<a href="https://colab.research.google.com/github/edutavr/electromech-conversion/blob/main/Eletromec_Desafio_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Importando dependências

In [3]:
import pandas as pd
import numpy as np
import math

## Funções auxiliares

In [24]:
# Inicializando os parâmetros de entrada, no caso
# W2 --> Potência de saída
# V2 -->  Tensão Secundária
# V1 --> Tensão Primária
def initialize_parameters(W2, V2, V1):

    if W2.isdigit() & V2.isdigit() & V1.isdigit():
        if int(W2) > 800:
            print('Potência de saída do transformador é maior do que 800 VA. Não é possível usar lâminas padronizadas')
            laminas_padrozinadas = False
        else:
            print('A potência de saída do transformador é menor do que 800 VA. É possível utilizar as lâminas padronizadas')
            laminas_padrozinadas = True
        return laminas_padrozinadas
    else:
        print('Os valores inseridos não são dígitos...')
        return

def first_and_second_current(W2,V2,V1):
    Is = (W2/V2)

    #A Pontência primária é calculada a partir do aumento em 10% da potência secundária
    W1 = 1.1 * W2
    Ip = (W1/V1)

    return Is,Ip

def conductor_section(current,W2):
    if W2 == 500:
        return current/3
    elif W2 > 500 & W2 <= 1000:
        return  current/2.5
    elif W2 > 1000 & W2 <= 3000:
        return current/2
    else:
        return "Potência fora do range limite recomendado"

def bitola(condutor_section):
    conductor_section = {
    "upper_limit": [53.476, 42.409, 33.362, 26.271, 21.152, 16.774, 13.303, 10.549, 8.366, 6.635, 5.262, 4.173, 3.309, 2.624, 2.081, 1.650, 1.309, 1.038, 0.823, 0.653, 0.518],
    "lower_limit": [42.409, 33.362, 26.271, 21.152, 16.774, 13.303, 10.549, 8.366, 6.635, 5.262, 4.173, 3.309, 2.624, 2.081, 1.650, 1.309, 1.038, 0.823, 0.653, 0.518, 0.411],
    "description": ["fio 0", "fio 1", "fio 2", "fio 3", "fio 4", "fio 5", "fio 6", "fio 7", "fio 8", "fio 9", "fio 10", "fio 11", "fio 12", "fio 13", "fio 14", "fio 15", "fio 16", "fio 17", "fio 18", "fio 19", "fio 20"]
    }
    df = pd.DataFrame(conductor_section)
    row = df[(df['lower_limit'] < condutor_section) & (df['upper_limit'] >= condutor_section)]
    if not row.empty:
        upper_limit = row.iloc[0]['upper_limit']
        description = row.iloc[0]['description']
        return [upper_limit, description]
    else:
        return []


def magnectic_section(potency, frequency,is_long_cable,is_two_primary_circuits=False,is_two_secondary_circuits=False):
    standard_cables = 7.5*(math.sqrt(potency/frequency))
    long_cables = 6.5*(math.sqrt(potency/frequency))
    if is_two_primary_circuits is False and is_two_secondary_circuits is False:
        if is_long_cable:
            return long_cables
        return standard_cables

    if is_two_primary_circuits is True and is_two_secondary_circuits is False:
        standard_cables = 7.5*(math.sqrt(1.25*potency/frequency))
        long_cables = 6*(math.sqrt(1.25*potency/frequency))

        if is_long_cable:
            return long_cables
        return standard_cables

    if is_two_primary_circuits is True and is_two_secondary_circuits is True:
        standard_cables = 7.5*(math.sqrt(1.5*potency/frequency))
        long_cables = 6*(math.sqrt(1.5*potency/frequency))

        if is_long_cable:
            return long_cables
        return standard_cables

    print('Inválido')
    return [0, 0]


def core_geometric_section_1(magnetic_section):
    return magnetic_section*1.1


def calculate_a_and_b_geometric_section(geometric_section,a):
    b = geometric_section/a
    return b

def core_geometric_section(length, width):
    return length*width

def core_magnetic_section(length, width):
    return core_geometric_section(length, width)/1.1

def calculate_turns_number_1(frequency, tension, core_ms):
    if(frequency == 50):
        return tension*(40/core_ms)
    #Frequency = 60Hz
    return tension*(33.5/core_ms)


def blades_qtd(b:float,acesita:float):
  return round((b*0.9)/acesita)

def blade_type(a):
    if a == 1.5:
        return "Lãmina tipo 0"
    elif a == 2:
        return "Lãmina tipo 1"
    elif a == 2.5:
        return "Lãmina tipo 2"
    elif a == 3:
        return "Lãmina tipo 3"
    elif a == 3.5:
        return "Lãmina tipo 4"
    elif a == 4:
        return "Lãmina tipo 5"
    elif a == 5:
        return "Lâmina tipo 6"
    else:
        return "Lãmina não encontrada"


def dimensions_core(a,b,second_potency):
    if(second_potency>800):
        largura=3*a
        altura=4*a
        comprimento=b
        seção_janela=(0.5*a*1.5*a)*100  ## mm²
        volume= ((largura*altura) - (0.5*a*3*a*2))*b*0.9 ## cm³
        peso = 5.4*(a**2)*7.8


        return ("\nA Largura é: "+str(largura)+"cm\n" +
                "A Altura é: "+str(altura)+"cm\n" +
                "O Comprimento é: "+str(comprimento)+"cm\n" +
                "A Seção da janela é: "+str(seção_janela)+"mm²\n"+
                "O Volume é: "+str(volume)+"cm³\n"
                "Peso é:" + str(peso)+ 'g')
    else:
        largura=3*a
        altura=2.5*a
        comprimento=b
        seção_janela=(0.5*a*1.5*a)*100  ## mm²
        volume= ((float(largura)*float(altura)) - (0.5*a*3*a*2))*b*0.9 ## cm
        peso = 5.4*(a**2)*7.8


        return ("\nA Largura é: "+str(largura)+"cm\n" +
                "A Altura é: "+str(altura)+"cm\n" +
                "O Comprimento é: "+str(comprimento)+"cm\n" +
                "A Seção da janela é: "+str(seção_janela)+"mm²\n"+
                "O Volume é: "+str(volume)+"cm³\n"
                "Peso é: " + str(peso)+ 'g')

## Parâmetros do transformador

In [25]:
W2 = 300
V1 = 220
V2 = 120
frequency = 50
espessura = 0.035

#Sistema

In [26]:
first_current, second_current = first_and_second_current(W2,V2,V1)

In [27]:
W1 = 1.1 * W2

In [28]:
section_1 = conductor_section(first_current, W2)
section_2 = conductor_section(second_current, W2)

In [31]:
section_1_bitola = bitola(section_1)
section_2_bitola = bitola(section_2)

In [32]:
magnetic_section = magnectic_section(W1,frequency,False)
geometric_section = core_geometric_section_1(magnetic_section)

In [33]:
if(W2>800):
  print("Uso das laminas Compridas pois a potencia é maior que 800VA")
else:
  print("Uso das laminas padronizadas pois a potencia é menor que 800VA")

Uso das laminas padronizadas pois a potencia é menor que 800VA


In [34]:
a = round(math.sqrt(geometric_section))
blade = blade_type(a)
b = round(calculate_a_and_b_geometric_section(geometric_section,a),1)

In [35]:
core_gs = core_geometric_section(a, b)
core_ms =round(core_magnetic_section(a, b),1)
n1 = calculate_turns_number_1(frequency, W1, core_ms)
n2 = calculate_turns_number_1(frequency,  W2, core_ms)*1.1

In [36]:
dimensions= dimensions_core(a,b,W2)
qtd_blades= blades_qtd(b,espessura)

In [37]:
print("Número de Espiras do Enrolamento Primário: ", n1)
print("Número de Espiras do Enrolamento Secundário: ", n2)
print("Bitola do cabo primário: ", section_1_bitola)
print("Bitola do cabo secundário: ", section_2_bitola)
print("Tipo de lâmina: ", blade)
print("Quantidade de lâminas: ", qtd_blades)
print("Dimensões: ",dimensions)

Número de Espiras do Enrolamento Primário:  691.0994764397906
Número de Espiras do Enrolamento Secundário:  691.0994764397907
Bitola do cabo primário:  [np.float64(1.038), 'fio 17']
Bitola do cabo secundário:  [np.float64(0.653), 'fio 19']
Tipo de lâmina:  Lâmina tipo 6
Quantidade de lâminas:  108
Dimensões:  
A Largura é: 15cm
A Altura é: 12.5cm
O Comprimento é: 4.2cm
A Seção da janela é: 1875.0mm²
O Volume é: 425.25cm³
Peso é: 1053.0g
